## Параметры

In [300]:
import copy
import random

In [301]:
ITERATIONS = 50
POPULATION_SIZE = 200

### Начальная особь - "Стеарат натрия"
###### SMILES:  CCCCCCCCCCCCCCCCCC(=O)[O-].[Na+]

In [302]:
# Спросить про правильность переовода в smiles - NaC18H35O2
base_individual = ['C','C','C','C','C','C','C','C','C','C','C','C','C','C','C','C','C','C(=O)[O-].[Na+]']

### Нужно получить - "Лаурилсульфат натрия"
###### SMILES:  CCCCCCCCCCCCOS(=O)([O-])=O.[Na+]

In [303]:
# Спросить про правильность переовода в smiles - C12H25 [SO4NA]
goal_individual = ['C','C','C','C','C','C','C','C','C','C','C','C','O','C','C','OS(=O)([O-])=O.[Na+]']

In [304]:
rating = {
    'OS(=O)([O-])=O.[Na+]': 38.7, # -SO4Na
    'C(=O)[O-].[K+]': 21.1,       # -COOK (правильное значение rating? или поменять местами с нижним)
    'C(=O)[O-].[Na+]': 19.1,      # -COONa
    'N': 9.4,                  # 
    'C(=O)[O-]': 2.1,             # -COOH
    'O': 1.9,                     # -OH
    'C': 0.5
}

In [305]:
head_elements = ['OS(=O)([O-])=O.[Na+]', 'C(=O)[O-].[K+]', 'C(=O)[O-].[Na+]', 'C(=O)[O-]']
body_elements = ['N', 'C', 'O']
tail_elements = ['C']

In [306]:
def mutate(old_individual, max_mutation_count=3):
    new_individual = copy.deepcopy(old_individual)
    mutations_count = random.randint(1, max_mutation_count)

    positions = []
    for i in range(mutations_count):
        positions.append(random.choice([0, len(new_individual)-1, random.randint(1, len(new_individual)-2)]))
    mutation_variants = [0, 1]
    
    for pos in positions:
#         print('before: {0}'.format(new_individual))
        if pos == 0:
            elem = random.choice(tail_elements)
            new_individual[pos] = elem
        elif pos == len(new_individual) - 1:
            elem = random.choice(head_elements)
            new_individual[pos] = elem
        else:
            mutation_vartiant = random.choice(mutation_variants)
            elem = random.choice(body_elements)
#             print('variant: {0}'.format(mutation_vartiant))
#             if mutation_vartiant == 0:
                
#                 new_individual[pos] = elem
#             else:
            new_individual.insert(pos, elem)
#         print('after: {0}'.format(new_individual))
    return new_individual         

In [307]:
# mutate(base_individual)

In [308]:
def score(individual):
    return sum([rating[elem] for elem in individual])

goal_score = score(goal_individual)

def delta_glb(individual):
    ind_score = score(individual)
    return abs(goal_score - ind_score)

def sort_by_score(population):
    res = []
    for individual in population:
        cond1 = individual[0] in tail_elements
        cond2 = individual[len(individual) - 1] in head_elements
        # cond3 =  # is poison?
        if cond1 and cond2:
            res.append((individual, score(individual), delta_glb(individual)))
        res = sorted(res, key=lambda el: el[2])
    return list(map(lambda individual: individual[0], res))[:POPULATION_SIZE]

def ind_to_str(individual):
    res = ''
    for el in individual:
        res += el
    return res

goal_score

47.6

In [309]:
def cross(individual_one, individual_two):
    new_individual_one = copy.deepcopy(individual_one)
    new_individual_two = copy.deepcopy(individual_two)

    pos1 = random.randint(1, len(individual_one) - 2)
    pos2 = random.randint(1, len(individual_two) - 2)
    
    buf = new_individual_one[pos1]
    new_individual_one[pos1] = new_individual_two[pos2]
    new_individual_two[pos2] = buf
    
    return new_individual_one, new_individual_two
 

def get_random_interval(individual):
    N = len(individual)
    start_pos = random.randint(1, N - 1)
    finish_pos = random.randint(1, N - 1)
    
    if start_pos > finish_pos:
        temp = start_pos
        start_pos = finish_pos
        finish_pos = temp
    elif start_pos == finish_pos:
        if finish_pos == N-1:
            start_pos -= 1
        else:
            finish_pos += 1
        
    return start_pos, finish_pos

    
def cross1(individual_one, individual_two):
    new_individual_one = copy.deepcopy(individual_one)
    new_individual_two = copy.deepcopy(individual_two)
    
#     start_pos1 = random.randint(1, len(individual_one) - 2)
#     finish_pos1 = random.randint(1, len(individual_one) - 2) - start_pos1
    start_pos1, finish_pos1 = get_random_interval(individual_one)
#     print('1) [{0}:{1}]'.format(start_pos1, finish_pos1))
#     print(individual_one[start_pos1:finish_pos1]) 

    start_pos2, finish_pos2 = get_random_interval(individual_two)
#     print('2) [{0}:{1}]'.format(start_pos2, finish_pos2))
#     print(individual_two[start_pos2:finish_pos2]) 


    buf = new_individual_one[start_pos1:finish_pos1]
    new_individual_one = new_individual_one[:start_pos1] + new_individual_two[start_pos2:finish_pos2] + new_individual_one[finish_pos1:]
    new_individual_two = new_individual_two[:start_pos2] + buf + new_individual_two[finish_pos2:]
    
    return new_individual_one, new_individual_two
 

In [310]:
ind1 = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1']
ind2 = ['A2', 'B2', 'C2', 'D2', 'E2', 'F2']
for pop in cross1(ind1, ind2):
    print(ind_to_str(pop))

A1B2E1F1
A2B1C1D1C2D2E2F2


In [311]:
ind1[3:5]

['D1', 'E1']

In [312]:
ind1[:3], ind1[3:5], ind1[5:]

(['A1', 'B1', 'C1'], ['D1', 'E1'], ['F1'])

In [313]:
ind2[:1], ind2[1:3], ind2[3:]

(['A2'], ['B2', 'C2'], ['D2', 'E2', 'F2'])

In [314]:
def generate(population):
    next_gen = []
    for individual in population:
        child1 = mutate(individual)
        child2, child3 = cross1(individual, child1)
        next_gen.extend([individual, child1, child2, child3])
    return copy.deepcopy(next_gen)

In [315]:
def run_ga(population):
    new_population = population
    for i in range(ITERATIONS):
        new_population = generate(new_population)
#         for pop in new_population:
#             print(ind_to_str(pop))
        new_population = sort_by_score(new_population)
    return new_population
ind_to_str(base_individual)

'CCCCCCCCCCCCCCCCCC(=O)[O-].[Na+]'

In [316]:
k = 0
print('[{0}] {1}'.format(k, ind_to_str(goal_individual)))
print('Score:\t\t{0}'.format(goal_score))
for individual in run_ga([base_individual])[:10]:
    k+=1
    print('[{0}] {1}'.format(k, ind_to_str(individual)))
    print('Score:\t\t{0}'.format(score(individual)))

[0] CCCCCCCCCCCCOCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[1] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[2] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[3] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[4] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[5] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[6] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[7] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[8] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[9] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
[10] CCCOCCCCCCCCCCCOS(=O)([O-])=O.[Na+]
Score:		47.6
